<a href="https://colab.research.google.com/github/JhonatanCaldeira/Simplon_ML_2024/blob/main/mistral.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip install -q datasets loralib sentencepiece
!pip -q install bitsandbytes accelerate xformers einops
# !pip -q install langchain

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 70.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 69.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 107.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12

In [3]:
!nvidia-smi

Tue Dec 12 13:53:23 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install autoawq

In [5]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [6]:
torch.set_default_device('cuda')

In [7]:
model_id = "mistralai/Mistral-7B-Instruct-v0.1"


bnb_config = transformers.BitsAndBytesConfig(
  load_in_4bit=True,
  bnb_4bit_use_double_quant=True,
  bnb_4bit_quant_type="nf4",
  bnb_4bit_compute_dtype=torch.bfloat16
)

model = transformers.AutoModelForCausalLM.from_pretrained(
  model_id,
  trust_remote_code=True,
  quantization_config=bnb_config,
  device_map='auto',
)

tokenizer = transformers.AutoTokenizer.from_pretrained(
model_id,
)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [9]:
text = "<s>[INST] What is your favourite condiment? [/INST]Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!</s> [INST] Do you have mayonnaise recipes? [/INST]"

encodeds = tokenizer(text, return_tensors="pt", add_special_tokens=False)

generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] What is your favourite condiment? [/INST]Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!</s>  [INST] Do you have mayonnaise recipes? [/INST] Yes, I have a classic mayonnaise recipe that you can make right in your kitchen. Here's what you'll need:

* 2 large egg yolks
* 2 tablespoons of Dijon mustard
* 1/4 cup of white wine vinegar
* 1/2 teaspoon of salt
* 1/3 cup of vegetable oil
* 1/4 cup of olive oil
* Optional: 1 tablespoon of finely chopped fresh herbs or spices (such as parsley, dill, or tarragon)

To make the mayonnaise, begin by whisking the egg yolks and mustard together in a small bowl until well combined. Add in the vinegar and salt, and whisk again until everything is evenly mixed.

Next, slowly add in the olive oil, whisking constantly, until the mixture thickens and becomes smooth and creamy. You may need to do this in stages, allowing each addition of oil t

### Prompt Format

```
# Isto está formatado como código
```
```
<s>[INST] <user_prompt> [/INST]

<assistant_response> </s>

[INST] <user_prompt>[/INST]
```

In [10]:
import textwrap

def wrap_text(text, width=90): #preserve_newlines
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

In [13]:
def generate(input_text, system_prompt="",max_length=512):
    prompt = f"""<s>[INST]{input_text}[/INST]"""
    inputs = tokenizer(input_text, return_tensors="pt", add_special_tokens=False)

    outputs = model.generate(**inputs,
                             max_length=max_length,
                             temperature=0.1,
                             do_sample=True)
    text = tokenizer.batch_decode(outputs)[0]
    wrapped_text = wrap_text(text)
    print(wrapped_text)

## CodeGen

In [14]:
generate('''```python
def print_prime(n):
   """
   Print all primes between 1 and n
   """''', system_prompt="You are a genius python coder")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


```python
def print_prime(n):
   """
   Print all primes between 1 and n
   """
   for i in range(2, n+1):
       if is_prime(i):
           print(i)

def is_prime(n):
   """
   Check if n is prime
   """
   if n <= 1:
       return False
   for i in range(2, int(n**0.5)+1):
       if n % i == 0:
           return False
   return True

print_prime(100)
```

This code defines two functions: `print_prime` and `is_prime`. The `print_prime` function
takes an integer `n` as input and prints all primes between 1 and `n`. The `is_prime`
function takes an integer `n` as input and returns `True` if `n` is prime, and `False`
otherwise.

The `print_prime` function uses a loop to iterate over all integers between 2 and `n`, and
checks if each integer is prime using the `is_prime` function. If an integer is prime, it
is printed to the console.

The `is_prime` function uses a loop to iterate over all integers between 2 and the square
root of `n`, and checks if `n` is divisible by any of these intege

## Instruction Answering

In [15]:
generate('Write a detailed analogy between mathematics and a lighthouse.',
         max_length=128)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Write a detailed analogy between mathematics and a lighthouse.

Mathematics is like a lighthouse in many ways. Just as a lighthouse guides ships safely to
shore by emitting a steady beam of light, mathematics provides a clear and precise set of
rules and principles that guide us through the complexities of the world.

The lighthouse is a towering structure that stands tall and unwavering, just as
mathematics is a solid foundation that we can rely on to guide us through life. The
lighthouse is also equipped with advanced technology, such as radar and sonar


## Chat

In [16]:
generate("""Alice: I don't know why, I'm struggling to maintain focus while studying. Any suggestion? \n\n Bob:""",
         max_length=128)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Alice: I don't know why, I'm struggling to maintain focus while studying. Any suggestion?

 Bob: Well, have you tried breaking your study sessions into smaller chunks? It's easier
to concentrate when you're studying for shorter periods.

 Alice: That's a good idea, I'll give it a try.

 Bob: Also, make sure you're taking breaks in between study sessions. It's important to
give your brain a break to recharge.

 Alice: I'll make sure to do that.

 Bob: And don't forget
